In [5]:
%load_ext swagpy.jupyter

In [36]:
from collections import namedtuple
from functools import wraps
from dataclasses import dataclass
from typing import TypeVar
T = TypeVar("T")

def get_members(cls: type):
    return {k: v for k, v in vars(cls).items() if not k.startswith("__")}


def frozen_singleton(cls: type[T]) -> T:
    members = get_members(cls)
    return namedtuple(cls.__name__, members.keys())(**members)


@frozen_singleton
class C:
    a = 1


C.a

<function C.count(value, /)>

In [22]:
import numpy as np
import pandas as pd

TOOO_SLOW = 100
idx = pd.IndexSlice
random = np.random.randint
lindex = lambda x1, x2, s, n: pd.Index(np.linspace(x1, x2, s, dtype=np.float32), name=n)


def make_gs(x1, y1, x2, y2, x_size, y_size):

    col = lindex(x1, x2, x_size, "lon")
    ndx = lindex(y1, y2, y_size, "lat")
    grd = (
        pd.DataFrame(columns=col, index=ndx).unstack("lat").reset_index().dropna(axis=1)
    )
    grd["wv"] = random(0, 255, size=len(grd))
    grd["ir"] = random(0, 255, size=len(grd))
    return grd.set_index(["lat", "lon"]).sort_index(level=["lat", "lon"])


min_diff = lambda t, v: t[np.argmin(abs(t[:, np.newaxis] - v), axis=0)]


def main():
    # GRIDSPACE
    gs = make_gs(-129, 54, -60, 20, x_size=972, y_size=635)
    lat, lon = (gs.index.unique(crd).to_numpy() for crd in ("lat", "lon"))

    # FEATURES
    n_features = 76_020
    f = pd.DataFrame(
        [
            {"minx": -126, "maxx": -68, "miny": 37, "maxy": 39},
            {"minx": -91, "maxx": -70, "miny": 31, "maxy": 37},
            {"minx": -124, "maxx": -64, "miny": 24, "maxy": 26},
        ]
        * (n_features // 3)
    )

    params = [(lon, "minx"), (lon, "maxx"), (lat, "miny"), (lat, "maxy")]
    arr = np.array([min_diff(a, f[col].to_numpy()) for a, col in params]).T
    result = [gs.loc[idx[x1:x2, y1:y2, :]] for x1, x2, y1, y2 in arr[:TOOO_SLOW]]
    # How can this loop be vectorized
    return result


if __name__ == "__main__":
    main()